In [70]:
import ee
import geemap
import os
import shapely
import json
import mgrs
import pandas as pd
import ipywidgets
import folium
import geemap.eefolium as gmap
import traceback

# Initialize Map Object

In [71]:
ee.Initialize()

In [3]:
Map = gmap.Map()

# Import Polygons

In [4]:
def import_aois(csv_loc):    

        df_labels = pd.read_csv(csv_loc)
        df_labels = df_labels[["center-lat","center-long","polygon","Labels combined"]]

        polygons = []
        for polygon in df_labels["polygon"]:
            polygons.append(json.loads(polygon)["coordinates"])


        tiles = []
        tiles_dic = {}
        polygon_id = 0 
        coordinates = []
        m = mgrs.MGRS()
        for items in polygons:
            polygon_id += 1 
            for item in items:
                for lon_lat in item:
                    coordinates.append(lon_lat)
                    c = m.toMGRS(lon_lat[1], lon_lat[0])
                    tile = c[0:5]
                    

                    if polygon_id in tiles_dic:

                        tiles_dic[polygon_id].append(tile)

                    else:

                        tiles_dic[polygon_id] = [tile]

                    tiles.append(tile)

                tiles_dic[polygon_id] = list(set(tiles_dic[polygon_id]))

        tiles = list(set(tiles))

        df_labels["tiles"] = tiles_dic.values()

        #bounding box

        min_lon = min([i[0] for i in coordinates])
        min_lat = min([i[1] for i in coordinates])
        max_lon = max([i[0] for i in coordinates])
        max_lat = max([i[1] for i in coordinates])

        bounding_box = min_lon,min_lat,max_lon,max_lat
        
        return polygons

In [5]:
polygons = import_aois("/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v3/Polygon_List/polygons_101320.csv")

/Users/purgatorid/opt/anaconda3/envs/gee-conda/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [6]:
Map = gmap.Map()

# start and end of time series
startDate = ee.Date('2020-01-01')
stopDate  = ee.Date('2020-12-31')
TCI_RGB = ['TCI_R', 'TCI_G', 'TCI_B']
count_loop = 0
count_bad = 0  
good_images = []
good_dict = {}
feature_id = 0 
features = []


for poly in polygons[0:3]:
    Map = gmap.Map()
    # create an roi. first item in Misha's label list
    feature_id += 1 
    
    # create geometry object, create feature object, append to features list for feature collection creation 
    polys = ee.Geometry.Polygon(poly)
    feature = ee.Feature(polys,{"name":feature_id})
    features.append(feature)
    
    centroid = polys.centroid()
    lng, lat = centroid.getInfo()['coordinates']
    print("lng = {}, lat = {}".format(lng, lat))
    Map.setCenter(lng, lat, 12)

    # Add Earth Engine dataset
    # Sentinel 2 L2A ImageCollection
    # filter by bounds and data
    collection = ee.ImageCollection('COPERNICUS/S2_SR')\
        .filterBounds(polys)\
        .filterDate(startDate,stopDate)\
        .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE",5))\
        .sort('CLOUD_COVER',True)\
        .select(TCI_RGB)
    

    # use only first image from collection
    img = ee.Image(collection.first())
    
    # clip image to the specific geometry
    img = img.clip(polys)

    # use the mean image - would require clipping image again
#     collection_mean = collection.mean()


    # print out the number of images in the ImageCollection
    count = collection.size().getInfo()
    print("Count: ", count)
    
    # viz params
    vis = {'bands': TCI_RGB, "gamma": [2, 2, 2]}
    
    count_loop += 1
    
    # if more than zero in the image collection
    if count > 0:
#         Map.addLayer(img,vis, f"Image ID {count_loop}")
        # Map.addLayer(collection,vis,"Original Collection")
        # Map.addLayer(collection_mean,vis, "Collection Mean")
        good_images.append(img)
        print(f"Added Image ID {count_loop}")
    
    else:
        count_bad += 1 
        continue

    

print(f'found {count_bad} polygons without a corresponding image collection')


# Create custom collection with list of images for DL
custom_collection = ee.ImageCollection.fromImages(good_images)
Map.addLayer(custom_collection, vis)
image_dl_list = custom_collection.toList(105)

# add all polygon features to map

fc = ee.FeatureCollection(features)
Map.add_layer(fc, {}, 'default display')

Map

lng = 9.163954053102769, lat = 5.7876496002257225
Count:  7
Added Image ID 1
lng = 8.920994498773, lat = 5.752791089641022
Count:  7
Added Image ID 2
lng = 13.198437170573584, lat = 5.597060007220655
Count:  17
Added Image ID 3
found 0 polygons without a corresponding image collection


In [16]:
RGB = ['TCI_R', 'TCI_G', 'TCI_B']

# Download the specific image
# for image_obj in range(len(good_images)):
for i in range(len(good_images)):
    downConfig = {'scale': 10, "maxPixels": 1.0E13, 'driveFolder': 'image3',"driveFileNamePrefix":str(i)}  # scale means resolution.
    image_to_dl = ee.Image(image_dl_list.get(i))
    # img_2 = image_to_dl.select('B.+')
    img_2 = image_to_dl.select(RGB)
    name = img_2.getInfo()["id"].split("/")[-1]
    print("Image to Download:",name)
    task = ee.batch.Export.image(img_2, name, downConfig)
    task.start()

while task.status()["state"] == 'RUNNING':
    print(task.status(), end="\r", flush=True)

Image to Download: 20200105T094309_20200105T095703_T32NNM
Image to Download: 20200105T094309_20200105T095703_T32NMM
Image to Download: 20200104T092401_20200104T093437_T33NTG
Image to Download: 20200104T092401_20200104T093437_T33NVF
Image to Download: 20200104T092401_20200104T093437_T33NTF
Image to Download: 20200105T094309_20200105T095703_T32NML
Image to Download: 20200104T092401_20200104T093437_T33NUF
Image to Download: 20200104T092401_20200104T093437_T33NVF
Image to Download: 20200107T093351_20200107T094442_T32NRL
Image to Download: 20200107T093351_20200107T094442_T32NRL
Image to Download: 20200114T092331_20200114T093418_T33NVE
Image to Download: 20200105T094309_20200105T095703_T32NML
Image to Download: 20200114T092331_20200114T093418_T33NVE
Image to Download: 20200213T092101_20200213T093321_T32NRK
Image to Download: 20200205T091049_20200205T091719_T33NWE
Image to Download: 20200205T091049_20200205T091719_T33NWE
Image to Download: 20200205T091049_20200205T091719_T33NXE
Image to Downl

# Sandbox

In [37]:
ee.Initialize()

In [44]:
def import_aois(csv_loc):    

        df_labels = pd.read_csv(csv_loc)
        df_labels = df_labels[["center-lat","center-long","polygon","Labels combined"]]

        polygons = []
        for polygon in df_labels["polygon"]:
            polygons.append(json.loads(polygon)["coordinates"])


        tiles = []
        tiles_dic = {}
        polygon_id = 0 
        coordinates = []
        m = mgrs.MGRS()
        for items in polygons:
            polygon_id += 1 
            for item in items:
                for lon_lat in item:
                    coordinates.append(lon_lat)
                    c = m.toMGRS(lon_lat[1], lon_lat[0])
                    tile = c[0:5]
                    

                    if polygon_id in tiles_dic:

                        tiles_dic[polygon_id].append(tile)

                    else:

                        tiles_dic[polygon_id] = [tile]

                    tiles.append(tile)

                tiles_dic[polygon_id] = list(set(tiles_dic[polygon_id]))

        tiles = list(set(tiles))

#         df_labels["tiles"] = tiles_dic.values()

        #bounding box

        min_lon = min([i[0] for i in coordinates])
        min_lat = min([i[1] for i in coordinates])
        max_lon = max([i[0] for i in coordinates])
        max_lat = max([i[1] for i in coordinates])

        bounding_box = min_lon,min_lat,max_lon,max_lat
        
        return polygons[0:1]

In [45]:
polygons = import_aois("/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v3/Polygon_List/polygons_101320.csv")

In [47]:
polygons

[[[[9.088783, 5.753152],
   [9.179249, 5.703106],
   [9.248257, 5.792946],
   [9.236069, 5.845887],
   [9.090672, 5.844009],
   [9.088783, 5.753152]]]]

In [81]:
# Map = gmap.Map()

# start and end of time series
startDate = ee.Date('2020-01-01')
stopDate  = ee.Date('2020-12-31')
TCI_RGB = ['TCI_R', 'TCI_G', 'TCI_B']
count_loop = 0
count_bad = 0  
good_images = []
good_dict = {}
feature_id = 0 
features = []
img_objs = []


for poly in polygons[0:1]:
#     Map = gmap.Map()
    # create an roi. first item in Misha's label list
    feature_id += 1 
    
    # create geometry object, create feature object, append to features list for feature collection creation 
    polys = ee.Geometry.Polygon(poly)
    feature = ee.Feature(polys,{"name":feature_id})
    features.append(feature)
    
    centroid = polys.centroid()
    lng, lat = centroid.getInfo()['coordinates']
    print("lng = {}, lat = {}".format(lng, lat))
    Map.setCenter(lng, lat, 12)
    
    
    sensor_list = ['COPERNICUS/S2','COPERNICUS/S2_SR']
    for sensor in sensor_list:
        # Add Earth Engine dataset
        # Sentinel 2 L2A ImageCollection
        # filter by bounds and data
        collection = ee.ImageCollection(sensor)\
            .filterBounds(polys)\
            .filterDate(startDate,stopDate)\
            .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE",5))\
            .sort('CLOUD_COVER',True)


        # use only first image from collection
        img = ee.Image(collection.first())

        # clip image to the specific geometry
        img = img.clip(polys)
        img_objs.append(img)

lng = 9.163954053102769, lat = 5.7876496002257225


In [101]:
for img in img_objs:
    dic = img.getInfo()
    print(json.dumps(str(dic)))
#     product_id = str(img.getInfo()["properties"]['PRODUCT_ID'])
#     print(product_id)
#     print([product_id.split("_")[1],product_id.split("_")[2], product_id.split("_")[-2]])
    

"{'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [295, 264], 'origin': [164, 897], 'crs': 'EPSG:32632', 'crs_transform': [60, 0, 499980, 0, -60, 700020]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [1767, 1580], 'origin': [984, 5384], 'crs': 'EPSG:32632', 'crs_transform': [10, 0, 499980, 0, -10, 700020]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [1767, 1580], 'origin': [984, 5384], 'crs': 'EPSG:32632', 'crs_transform': [10, 0, 499980, 0, -10, 700020]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [1767, 1580], 'origin': [984, 5384], 'crs': 'EPSG:32632', 'crs_transform': [10, 0, 499980, 0, -10, 700020]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [88

In [57]:
mosaic = custom_collection.mosaic().clip(polys)

In [60]:
RGB = ['TCI_R', 'TCI_G', 'TCI_B']

# Download the specific image
# for image_obj in range(len(good_images)):
downConfig = {'scale': 10, "maxPixels": 1.0E13, 'driveFolder': 'image3',"driveFileNamePrefix":"mosaic"}  # scale means resolution.
# image_to_dl = ee.Image(image_dl_list.get(i))
# img_2 = image_to_dl.select('B.+')
# img_2 = image_to_dl.select(RGB)
name = "mosaic"
print("Image to Download:",name)
task = ee.batch.Export.image(mosaic, name, downConfig)
task.start()

while task.status()["state"] == 'RUNNING':
    print(task.status(), end="\r", flush=True)

Image to Download: mosaic


In [63]:
while task.status()["state"] == 'RUNNING' or task.status()["state"] == 'READY':
    print(task.status(), end="\r", flush=True)